# GPU-Accelerated Spark Connect Demo - ETL and ML Pipeline (Spark 4.0+)

Based on the Data and AI Summit 2025 session: [GPU Accelerated Spark Connect](https://www.databricks.com/dataaisummit/session/gpu-accelerated-spark-connect)


## Connect to Spark via Spark Connect


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# Create GPU-accelerated Spark session using Spark Connect 4.0+
spark = (
  SparkSession.builder
    .remote('sc://spark-connect-server')
    .appName('GPU-Accelerated-ETL-ML-Demo') 
    .getOrCreate()
)

print(f'Spark version: {spark.version}')

Spark version: 4.0.0


## Smoke Test GPU


In [2]:
df = (
  spark.range(2 ** 35)
    .withColumn('mod10', col('id') % lit(10))
    .groupBy('mod10').agg(count('*'))
    .orderBy('mod10')
)
df.show()
# workaround to get a plan with GpuOverrides applied by disabling adaptive execution
spark.conf.set('spark.sql.adaptive.enabled', False)
df.explain(mode='formatted')
spark.conf.set('spark.sql.adaptive.enabled', True)

+-----+----------+
|mod10|  count(1)|
+-----+----------+
|    0|3435973837|
|    1|3435973837|
|    2|3435973837|
|    3|3435973837|
|    4|3435973837|
|    5|3435973837|
|    6|3435973837|
|    7|3435973837|
|    8|3435973836|
|    9|3435973836|
+-----+----------+

== Physical Plan ==
GpuColumnarToRow (10)
+- GpuSort (9)
   +- GpuShuffleCoalesce (8)
      +- GpuColumnarExchange (7)
         +- GpuHashAggregate (6)
            +- GpuShuffleCoalesce (5)
               +- GpuColumnarExchange (4)
                  +- GpuHashAggregate (3)
                     +- GpuProject (2)
                        +- GpuRange (1)


(1) GpuRange
Output [1]: [id#1L]
Arguments: 0, 34359738368, 1, 64, [id#1L], 536870912

(2) GpuProject
Input [1]: [id#1L]
Arguments: [(id#1L % 10) AS mod10#36L], true

(3) GpuHashAggregate
Input [1]: [mod10#36L]
Keys [1]: [mod10#36L]
Functions [1]: [partial_gpucount(1, false)]
Aggregate Attributes [1]: [count#39L]
Results [2]: [mod10#36L, count#40L]
Lore: 

(4) GpuColumnarExch

##  Should GPU Be Used from the next cell on?


In [3]:
accelerate_on_gpu = True

### ETL on GPU?

In [4]:
spark.conf.set('spark.rapids.sql.enabled', accelerate_on_gpu)  

### ML on GPU?

In [5]:
if accelerate_on_gpu:
  spark.conf.set('spark.connect.ml.backend.classes', 'com.nvidia.rapids.ml.Plugin')
else:
  spark.conf.unset('spark.connect.ml.backend.classes')

## Normalize references to the same bank 

In [6]:
import csv
with open('work/name_mapping.csv', 'r') as name_mapping_file:
  nm_reader = csv.reader(name_mapping_file,)
  name_mapping = [r for r in nm_reader]
name_mapping_df = spark.createDataFrame(name_mapping, ['from_seller_name', 'to_seller_name'])

(
  name_mapping_df
    .where(col('to_seller_name') == 'Wells Fargo' )
    .show(truncate=False)
)

+------------------------------------------------------+--------------+
|from_seller_name                                      |to_seller_name|
+------------------------------------------------------+--------------+
|WELLS FARGO CREDIT RISK TRANSFER SECURITIES TRUST 2015|Wells Fargo   |
|WELLS FARGO BANK,  NA                                 |Wells Fargo   |
|WELLS FARGO BANK, N.A.                                |Wells Fargo   |
|WELLS FARGO BANK, NA                                  |Wells Fargo   |
+------------------------------------------------------+--------------+



In [7]:
# String columns
cate_col_names = [
  'orig_channel',
  'first_home_buyer',
  'loan_purpose',
  'property_type',
  'occupancy_status',
  'property_state',
  'product_type',
  'relocation_mortgage_indicator',
  'seller_name',
  'mod_flag'
]
# Numeric columns
label_col_name = 'delinquency_12'
numeric_col_names = [
  'orig_interest_rate',
  'orig_upb',
  'orig_loan_term',
  'orig_ltv',
  'orig_cltv',
  'num_borrowers',
  'dti',
  'borrower_credit_score',
  'num_units',
  'zip',
  'mortgage_insurance_percent',
  'current_loan_delinquency_status',
  'current_actual_upb',
  'interest_rate',
  'loan_age',
  'msa',
  'non_interest_bearing_upb',
  label_col_name
]
all_col_names = cate_col_names + numeric_col_names

## Define ETL Process

### Functions to read raw columns

In [8]:
def read_raw_csv(spark, path):
  def _get_quarter_from_csv_file_name():
    return substring_index(substring_index(input_file_name(), '.', 1), '/', -1)

  with open('work/csv_raw_schema.ddl', 'r') as f:
    _csv_raw_schema_str = f.read()
  
  return (
    spark.read
      .format('csv') 
      .option('nullValue', '') 
      .option('header', False) 
      .option('delimiter', '|') 
      .schema(_csv_raw_schema_str) 
      .load(path) 
      .withColumn('quarter', _get_quarter_from_csv_file_name())
  )

def extract_perf_columns(rawDf):
  perfDf = rawDf.select(
    col('loan_id'),
    date_format(to_date(col('monthly_reporting_period'),'MMyyyy'), 'MM/dd/yyyy').alias('monthly_reporting_period'),
    upper(col('servicer')).alias('servicer'),
    col('interest_rate'),
    col('current_actual_upb'),
    col('loan_age'),
    col('remaining_months_to_legal_maturity'),
    col('adj_remaining_months_to_maturity'),
    date_format(to_date(col('maturity_date'),'MMyyyy'), 'MM/yyyy').alias('maturity_date'),
    col('msa'),
    col('current_loan_delinquency_status'),
    col('mod_flag'),
    col('zero_balance_code'),
    date_format(to_date(col('zero_balance_effective_date'),'MMyyyy'), 'MM/yyyy').alias('zero_balance_effective_date'),
    date_format(to_date(col('last_paid_installment_date'),'MMyyyy'), 'MM/dd/yyyy').alias('last_paid_installment_date'),
    date_format(to_date(col('foreclosed_after'),'MMyyyy'), 'MM/dd/yyyy').alias('foreclosed_after'),
    date_format(to_date(col('disposition_date'),'MMyyyy'), 'MM/dd/yyyy').alias('disposition_date'),
    col('foreclosure_costs'),
    col('prop_preservation_and_repair_costs'),
    col('asset_recovery_costs'),
    col('misc_holding_expenses'),
    col('holding_taxes'),
    col('net_sale_proceeds'),
    col('credit_enhancement_proceeds'),
    col('repurchase_make_whole_proceeds'),
    col('other_foreclosure_proceeds'),
    col('non_interest_bearing_upb'),
    col('principal_forgiveness_upb'),
    col('repurchase_make_whole_proceeds_flag'),
    col('foreclosure_principal_write_off_amount'),
    col('servicing_activity_indicator'),
    col('quarter')
  )
  return perfDf.select('*').filter('current_actual_upb != 0.0')

def extract_acq_columns(rawDf):
  acqDf = rawDf.select(
    col('loan_id'),
    col('orig_channel'),
    upper(col('seller_name')).alias('seller_name'),
    col('orig_interest_rate'),
    col('orig_upb'),
    col('orig_loan_term'),
    date_format(to_date(col('orig_date'),'MMyyyy'), 'MM/yyyy').alias('orig_date'),
    date_format(to_date(col('first_pay_date'),'MMyyyy'), 'MM/yyyy').alias('first_pay_date'),
    col('orig_ltv'),
    col('orig_cltv'),
    col('num_borrowers'),
    col('dti'),
    col('borrower_credit_score'),
    col('first_home_buyer'),
    col('loan_purpose'),
    col('property_type'),
    col('num_units'),
    col('occupancy_status'),
    col('property_state'),
    col('zip'),
    col('mortgage_insurance_percent'),
    col('product_type'),
    col('coborrow_credit_score'),
    col('mortgage_insurance_type'),
    col('relocation_mortgage_indicator'),
    dense_rank().over(Window.partitionBy('loan_id').orderBy(to_date(col('monthly_reporting_period'),'MMyyyy'))).alias('rank'),
    col('quarter')
  )

  return acqDf.select('*').filter(col('rank')==1)

### Define function to parse date in Performance data 

In [9]:
def _parse_dates(perf):
  return (
    perf
      .withColumn("monthly_reporting_period", to_date(col("monthly_reporting_period"), "MM/dd/yyyy")) 
      .withColumn("monthly_reporting_period_month", month(col("monthly_reporting_period"))) 
      .withColumn("monthly_reporting_period_year", year(col("monthly_reporting_period"))) 
      .withColumn("monthly_reporting_period_day", dayofmonth(col("monthly_reporting_period"))) 
      .withColumn("last_paid_installment_date", to_date(col("last_paid_installment_date"), "MM/dd/yyyy")) 
      .withColumn("foreclosed_after", to_date(col("foreclosed_after"), "MM/dd/yyyy")) 
      .withColumn("disposition_date", to_date(col("disposition_date"), "MM/dd/yyyy")) 
      .withColumn("maturity_date", to_date(col("maturity_date"), "MM/yyyy")) 
      .withColumn("zero_balance_effective_date", to_date(col("zero_balance_effective_date"), "MM/yyyy"))
  )

### Define function to create deliquency data frame from Performance data.  

The computed `delinquency_12` column denotes whether a loan will become delinquent by 3, 6, or 9 months, 
or not delinquent, within the next 12 month period.   

It will be the target label for ML multi-class prediction.

In [10]:
def _create_perf_deliquency(spark, perf):
    aggDF = perf.select(
            col("quarter"),
            col("loan_id"),
            col("current_loan_delinquency_status"),
            when(col("current_loan_delinquency_status") >= 1, col("monthly_reporting_period")).alias("delinquency_30"),
            when(col("current_loan_delinquency_status") >= 3, col("monthly_reporting_period")).alias("delinquency_90"),
            when(col("current_loan_delinquency_status") >= 6, col("monthly_reporting_period")).alias("delinquency_180")) \
            .groupBy("quarter", "loan_id") \
            .agg(
                max("current_loan_delinquency_status").alias("delinquency_12"),
                min("delinquency_30").alias("delinquency_30"),
                min("delinquency_90").alias("delinquency_90"),
                min("delinquency_180").alias("delinquency_180")) \
            .select(
                col("quarter"),
                col("loan_id"),
                (col("delinquency_12") >= 1).alias("ever_30"),
                (col("delinquency_12") >= 3).alias("ever_90"),
                (col("delinquency_12") >= 6).alias("ever_180"),
                col("delinquency_30"),
                col("delinquency_90"),
                col("delinquency_180"))
    #aggDF.printSchema()
    joinedDf = perf \
            .withColumnRenamed("monthly_reporting_period", "timestamp") \
            .withColumnRenamed("monthly_reporting_period_month", "timestamp_month") \
            .withColumnRenamed("monthly_reporting_period_year", "timestamp_year") \
            .withColumnRenamed("current_loan_delinquency_status", "delinquency_12") \
            .withColumnRenamed("current_actual_upb", "upb_12") \
            .select("quarter", "loan_id", "timestamp", "delinquency_12", "upb_12", "timestamp_month", "timestamp_year") \
            .join(aggDF, ["loan_id", "quarter"], "left_outer")

    # calculate the 12 month delinquency and upb values
    months = 12
    monthArray = [lit(x) for x in range(0, 12)]
    
    testDf = joinedDf \
            .withColumn("month_y", explode(array(monthArray))) \
            .select(
                    col("quarter"),
                    floor(((col("timestamp_year") * 12 + col("timestamp_month")) - 24000) / months).alias("josh_mody"),
                    floor(((col("timestamp_year") * 12 + col("timestamp_month")) - 24000 - col("month_y")) / months).alias("josh_mody_n"),
                    col("ever_30"),
                    col("ever_90"),
                    col("ever_180"),
                    col("delinquency_30"),
                    col("delinquency_90"),
                    col("delinquency_180"),
                    col("loan_id"),
                    col("month_y"),
                    col("delinquency_12"),
                    col("upb_12")) \
            .groupBy("quarter", "loan_id", "josh_mody_n", "ever_30", "ever_90", "ever_180", "delinquency_30", "delinquency_90", "delinquency_180", "month_y") \
            .agg(max("delinquency_12").alias("delinquency_12"), min("upb_12").alias("upb_12")) \
            .withColumn("timestamp_year", floor((lit(24000) + (col("josh_mody_n") * lit(months)) + (col("month_y") - 1)) / lit(12))) \
            .selectExpr("*", "pmod(24000 + (josh_mody_n * {}) + month_y, 12) as timestamp_month_tmp".format(months)) \
            .withColumn("timestamp_month", when(col("timestamp_month_tmp") == lit(0), lit(12)).otherwise(col("timestamp_month_tmp"))) \
            .withColumn("delinquency_12", ((col("delinquency_12") > 9).cast("int") + (col("delinquency_12") > 6).cast("int") + (col("delinquency_12") > 3).cast("int") + (col("upb_12") == 0).cast("int")).alias("delinquency_12")) \
            .drop("timestamp_month_tmp", "josh_mody_n", "month_y")

    return perf.withColumnRenamed("monthly_reporting_period_month", "timestamp_month") \
            .withColumnRenamed("monthly_reporting_period_year", "timestamp_year") \
            .join(testDf, ["quarter", "loan_id", "timestamp_year", "timestamp_month"], "left") \
            .drop("timestamp_year", "timestamp_month")

### Define function to create acquisition data frame from Acquisition data

In [11]:
def _create_acquisition(spark, acq):
    return acq.join(name_mapping_df, col("seller_name") == col("from_seller_name"), "left") \
      .drop("from_seller_name") \
      .withColumn("old_name", col("seller_name")) \
      .withColumn("seller_name", coalesce(col("to_seller_name"), col("seller_name"))) \
      .drop("to_seller_name") \
      .withColumn("orig_date", to_date(col("orig_date"), "MM/yyyy")) \
      .withColumn("first_pay_date", to_date(col("first_pay_date"), "MM/yyyy")) 

### Define Casting Process


This part is casting String column to Numeric one. 
Example:
```
col_1
 "a"
 "b"
 "c"
 "a"
# After String ====> Numeric
col_1
 0
 1
 2
 0
```  

### Define function to get column dictionary

Example

```
col1 = [row(data="a",id=0), row(data="b",id=1)]
```

In [12]:
def _gen_dictionary(etl_df, col_names):
    cnt_table = etl_df.select(posexplode(array([col(i) for i in col_names])))\
                    .withColumnRenamed("pos", "column_id")\
                    .withColumnRenamed("col", "data")\
                    .filter("data is not null")\
                    .groupBy("column_id", "data")\
                    .count()
    windowed = Window.partitionBy("column_id").orderBy(desc("count"))
    return cnt_table.withColumn("id", row_number().over(windowed)).drop("count")

### Define function to convert string columns to numeric



In [13]:
def _cast_string_columns_to_numeric(spark, input_df):
    cached_dict_df = _gen_dictionary(input_df, cate_col_names).cache()
    output_df = input_df
    #  Generate the final table with all columns being numeric.
    for col_pos, col_name in enumerate(cate_col_names):
        col_dict_df = cached_dict_df.filter(col("column_id") == col_pos)\
                                    .drop("column_id")\
                                    .withColumnRenamed("data", col_name)
        
        output_df = output_df.join(broadcast(col_dict_df), col_name, "left")\
                        .drop(col_name)\
                        .withColumnRenamed("id", col_name)
    return output_df     

### Define Main Function

In this function:
1. Parse date in Performance data by calling _parse_dates (parsed_perf)
2. Create deliqency dataframe(perf_deliqency) form Performance data by calling _create_perf_deliquency
3. Create cleaned acquisition dataframe(cleaned_acq) from Acquisition data by calling _create_acquisition
4. Join deliqency dataframe(perf_deliqency) and cleaned acquisition dataframe(cleaned_acq), get clean_df
5. Cast String column to Numeric in clean_df by calling _cast_string_columns_to_numeric, get casted_clean_df
6. Return casted_clean_df as final result

In [14]:
def run_mortgage(spark, perf, acq):
    parsed_perf = _parse_dates(perf)
    perf_deliqency = _create_perf_deliquency(spark, parsed_perf)
    cleaned_acq = _create_acquisition(spark, acq)
    clean_df = perf_deliqency.join(cleaned_acq, ["loan_id", "quarter"], "inner").drop("quarter")
    casted_clean_df = _cast_string_columns_to_numeric(spark, clean_df)\
                    .select(all_col_names)\
                    .withColumn(label_col_name, when(col(label_col_name) > 0, col(label_col_name)).otherwise(0))\
                    .fillna(float(0))
    return casted_clean_df

### Run ETL Pipeline

#### Read Raw Data and Run ETL Process, Save the Result

Convert CSV to Parquet

In [15]:
rawDf = read_raw_csv(spark, '/data/input.csv')
rawDf.write.parquet('/opt/spark/work-dir/parquet', mode='overwrite')

SparkException: Job aborted.

JVM stacktrace:
org.apache.spark.SparkException
	at org.apache.spark.sql.rapids.GpuFileFormatWriterBase.writeAndCommit(GpuFileFormatWriter.scala:316)
	at org.apache.spark.sql.rapids.GpuFileFormatWriterBase.executeWrite(GpuFileFormatWriter.scala:331)
	at org.apache.spark.sql.rapids.GpuFileFormatWriterBase.executeWrite$(GpuFileFormatWriter.scala:323)
	at org.apache.spark.sql.rapids.GpuFileFormatWriter$.executeWrite(GpuFileFormatWriter.scala:493)
	at org.apache.spark.sql.rapids.GpuFileFormatWriterBase.write(GpuFileFormatWriter.scala:197)
	at org.apache.spark.sql.rapids.GpuFileFormatWriterBase.write$(GpuFileFormatWriter.scala:83)
	at org.apache.spark.sql.rapids.GpuFileFormatWriter$.write(GpuFileFormatWriter.scala:493)
	at org.apache.spark.sql.rapids.GpuInsertIntoHadoopFsRelationCommand.runColumnar(GpuInsertIntoHadoopFsRelationCommand.scala:190)
	at com.nvidia.spark.rapids.GpuDataWritingCommandExec.sideEffectResult$lzycompute(GpuDataWritingCommandExec.scala:125